In [194]:
import pandas as pd

In [195]:
prices = pd.read_csv('prices.log', header=None, names=["Price", "Timestamp"])
predictions = pd.read_csv('predictions.log', header=None, names=["Price", "Timestamp"])

In [196]:
prices

,Price,Timestamp
0,3062.35,2025-01-28T18:37:42.0000000
1,3063.87,2025-01-28T18:37:27.0000000
2,3063.76,2025-01-28T18:37:12.0000000
3,3061.83,2025-01-28T18:36:57.0000000
4,3061.11,2025-01-28T18:36:41.0000000
...,...,...
1135,3135.49,2025-01-28T23:22:57.0000000
1136,3136.47,2025-01-28T23:22:42.0000000
1137,3135.61,2025-01-28T23:22:27.0000000
1138,3134.45,2025-01-28T23:22:11.0000000


In [197]:
predictions

,Price,Timestamp
0,3063.60,2025-01-28T18:38:59.1513230-05:00
1,3063.31,2025-01-28T18:41:30.7964530-05:00
2,3066.06,2025-01-28T18:44:03.7681660-05:00
3,3068.63,2025-01-28T18:46:36.0442070-05:00
4,3073.59,2025-01-28T18:49:07.9749490-05:00
...,...,...
109,3132.70,2025-01-28T23:15:22.2094130-05:00
110,3135.38,2025-01-28T23:17:54.5741660-05:00
111,3136.43,2025-01-28T23:20:26.1707640-05:00
112,3136.67,2025-01-28T23:22:58.4644890-05:00


In [198]:
try:
    prices['Timestamp'] = pd.to_datetime(prices['Timestamp']).dt.tz_localize('US/Eastern')
    predictions['Timestamp'] = pd.to_datetime(predictions['Timestamp']).dt.tz_localize('US/Eastern')
except: 
    print("conversion error. continuing...")

prices['Timestamp'] = pd.to_datetime(prices['Timestamp']).dt.tz_convert('UTC')
predictions['Timestamp'] = pd.to_datetime(predictions['Timestamp']).dt.tz_convert('UTC')

conversion error. continuing...


In [208]:
# Sort both DataFrames by Timestamp
prices = prices.sort_values('Timestamp')
predictions = predictions.sort_values('Timestamp')

# Perform the merge with a 2-second tolerance
merged_df = pd.merge_asof(
    prices,
    predictions,
    on='Timestamp',
    tolerance=pd.Timedelta(seconds=2),
    direction='nearest',  # Options: 'backward', 'forward', 'nearest'
    suffixes=('_real', '_pred')
)

# Display the merged DataFrame
df = merged_df.dropna()
df

,Price_real,Timestamp,Price_pred
14,3062.13,2025-01-28 23:38:58+00:00,3063.60
34,3066.72,2025-01-28 23:44:02+00:00,3066.06
44,3070.12,2025-01-28 23:46:35+00:00,3068.63
54,3072.93,2025-01-28 23:49:06+00:00,3073.59
84,3079.54,2025-01-28 23:56:44+00:00,3078.93
...,...,...,...
1044,3130.36,2025-01-29 04:00:09+00:00,3133.96
1054,3124.34,2025-01-29 04:02:42+00:00,3131.91
1064,3128.55,2025-01-29 04:05:12+00:00,3128.67
1114,3134.50,2025-01-29 04:17:53+00:00,3135.38


##### Calculate % Error

In [202]:
# Calculate % Difference
df['% Difference'] = ((df['Price_real'] - df['Price_pred']) / df['Price_pred']) * 100

# Calculate the average of the absolute values
average_abs = df['% Difference'].abs().mean()
print(f"Average difference: {average_abs}%")

Average difference: 0.08923470165211292%


/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_21837/72705116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['% Difference'] = ((df['Price_real'] - df['Price_pred']) / df['Price_pred']) * 100


##### Calculate % Correct BUY/SELL predictions

In [209]:
prices_prices = list(prices['Price'])
prices_timestamp = list(prices['Timestamp'])
prediction_timestamps = list(df['Timestamp'])
predictions_prices = list(df['Price_pred'])

In [216]:
correct_side = 0
incorrect_side = 0
buy_predictions = 0
sell_predictions = 0
for i in range(len(prediction_timestamps)):
    timestamp = prediction_timestamps[i]

    # Find index of real price
    index = prices_timestamp.index(timestamp)
    # Real price of ETH
    real_price = prices_prices[index]
    #Predicted price of ETH
    predicted_price = predictions_prices[i]
    if index - 5 < 0:
        continue
    #Price 5 timestamps ago
    previous_price = prices_prices[index - 5]

    #Check if both prediction and real price were up or down
    if (real_price - previous_price > 0 and predicted_price - previous_price > 0):
        correct_side += 1
        buy_predictions += 1
    elif (real_price - previous_price < 0 and predicted_price - previous_price < 0):
        correct_side += 1
        sell_predictions += 1
    else:
        incorrect_side += 1

print(f"Correct Predictions: {correct_side} Incorrect Predictions: {incorrect_side}")
print(f"Correct Buy Predictions: {buy_predictions} Correct Sell Predictions: {sell_predictions}")

Correct Predictions: 42 Incorrect Predictions: 28
Correct Buy Predictions: 41 Correct Sell Predictions: 1
